### 향수 이름과 링크가 들어있는 파일 불러오기

In [1]:
import pandas as pd

df = pd.read_csv('basenote.csv', names=['name', 'url'], encoding='utf-8')


### 필요한 패키지 import

In [2]:
import requests
import time
from bs4 import BeautifulSoup
import csv
import time
import urllib.request

### 크롤링 결과를 저장할 배열

In [3]:
base_perfumes = ["no", "perfume", "brand", "gender", "avg_rating", "review_count", "positive", "neutral", "negative", "describe", "top_notes", "middle_notes", "base_notes"]
base_reviews = [["user_name", "no", "perfume", "brand", "review"]]

# with open('basenote_perfumes.csv', mode='a', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(base_perfumes)

# with open('basenote_reviews.csv', mode='a', newline='', encoding='utf-8') as f:
#     writer = csv.writer(f)
#     for row in base_reviews:
#         writer.writerow(row)

### 크롤링 시작페이지, 끝페이지 지정

In [4]:
start = 31156
end = 33000

In [5]:
for n in range(start, end):
    # try 안에서 에러나면 다음꺼로 넘어감
    try:
        # URL 설정
        url = df.url[n]
        
        no = n+1

        # User-Agent 헤더 설정
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"}

        # 페이지 요청
        response = requests.get(url, headers=headers)

        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(response.text, "html.parser")

        perfume = soup.find_all("span", {"itemprop": "Name"})[0].text.strip()
        brand = soup.find_all("span", {"itemprop": "brand"})[0].text.strip()
        gender = soup.find("span", {"class": "font-light"}).find('i').get('class')[0]

        rating = soup.find_all("div", {"class": "bntopdeets"})[0].find_all("span")[1]
        averageRating = 0
        for i_tag in rating.find_all('i'):
            for class_name in i_tag.get('class'):
                if(class_name != 'fas'): continue
                averageRating = averageRating + 1

        reviewCountTemp = soup.find_all("div", {"class": "bntopdeets"})[0].find_all("span")
        reviewCount = 0 if len(reviewCountTemp) == 2 else reviewCountTemp[2].text.split()[0]

        describe = soup.find_all("div", {"class" : "bnflexfluid3"})[0].find_all("p")[0].text.strip().replace(u'\xa0',u'').replace(u'\n',u'')

        allNotes = soup.find_all("ol", {"class" : "fragrancenotes"})[0].find_all("ul")
        notes = [[],[],[]]
        i = 0
        for note in allNotes:
          test = note.find_all('li')
          for list in test:
            notes[i].append(list.text.strip())
          i = i + 1 

        temp1 = soup.find("div", {"style": "width:80%;margin-left: auto;margin-right: auto;"}).find_all("a")[0].text.split()
        temp2 = soup.find("div", {"style": "width:80%;margin-left: auto;margin-right: auto;"}).find_all("a")[1].text.split()
        temp3 = soup.find("div", {"style": "width:80%;margin-left: auto;margin-right: auto;"}).find_all("a")[2].text.split()
        positive = 0 if temp1 == [] else temp1[0]
        neutral = 0 if temp2 == [] else temp2[0]
        negative = 0 if temp3 == [] else temp3[0]

        reviews = []
        for data in soup.find_all("div", {"class": ["reviewbox rthumbup", "reviewbox rthumbneutral", "reviewbox rthumbdown"]}):
          name = data.find("div", {"class": "reviewname"}).find("b").text
          review = data.find("div", {"class": ["reviewthumb thumbup", "reviewthumb thumbneutral", "reviewthumb thumbdown"]}).get("title")

          temp = name + ", " + review
          # 리뷰 결과
          reviews.append([name, no, perfume, brand, review])


        # 향수 결과
        result = [no, perfume, brand, gender, averageRating, reviewCount, positive, neutral, negative, describe, ', '.join(notes[0]), ', '.join(notes[1]), ', '.join(notes[2])]

        #csv 파일로 저장
        with open('basenote_perfumes.csv', mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(result)

        with open('basenote_reviews.csv', mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for row in reviews:
                writer.writerow(row)
        
        # 이미지 저장
        img_url = soup.find('img', {'class': 'bnheroimage'})['src']
        urllib.request.urlretrieve("https://basenotes.com"+img_url, "img/" + str(n) + ".png")
        
        

        print("========================================",no , "of", end, "====================================================")
        print("Name \t\t: " + perfume)
        print("Brand \t\t: " + brand)
        print("Gender \t\t: " + gender)
        print("Average_Rating \t:", averageRating)
        print("Review_Count \t:", reviewCount)
        print("Positive \t:", positive)
        print("Neutral \t:", neutral)
        print("Negative \t:", negative)
        print("Describe \t:", describe[:100], "...")
        print("Top_Notes \t:", notes[0])
        print("Middle_Notes \t:", notes[1])
        print("Base_Notes \t:", notes[2])
        print("Reviews \t:", reviews)
        print()


        time.sleep(3)
    except Exception as e:
        print("========================================",no , "of", end, "====================================================")
        print("********************error******************")
        print()
        continue

======================================== 31157 of 56640 ====================================================
********************error******************

======================================== 31158 of 56640 ====================================================
Name 		: Lancetti Pour Homme (original)
Brand 		: Lancetti
Gender 		: genderM
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Lancetti Pour Homme (original) is a men's fragrance launched in 1999 by Lancetti ...
Top_Notes 	: ['bergamot', 'grapefruit', 'thyme', 'violet leaf']
Middle_Notes 	: ['jasmine', 'rose', 'geranium', 'nutmeg', 'bay leaf', 'bourbon']
Base_Notes 	: ['patchouli', 'sandalwood', 'musk', 'cedarwood', 'amber', 'leather']
Reviews 	: []

======================================== 31159 of 56640 ====================================================
Name 		: Earthtones #5 : Tropical Lagoon
Brand 		: Neil Morris Fragrances
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 4


======================================== 31177 of 56640 ====================================================
Name 		: Flores Mediterráneas : Verbena y Azahar
Brand 		: Alvarez Gomez
Gender 		: genderS
Average_Rating 	: 3
Review_Count 	: 1
Positive 	: 1
Neutral 	: 0
Negative 	: 0
Describe 	: Flores Mediterráneas : Verbena y Azahar is a shared scent launched in 2017 by Alvarez Gomez ...
Top_Notes 	: ['lemon', 'lime', 'orange', 'tangerine', 'verbena', 'thyme', 'ginger']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: [['metrico', 31177, 'Flores Mediterráneas : Verbena y Azahar', 'Alvarez Gomez', 'Positive']]

======================================== 31178 of 56640 ====================================================
Name 		: A : Green Cachemire [The Red Series]
Brand 		: Blood Concept
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: A : Green Cachemire [The Red Series] is a shared scent launched in 2015 by Blood Concept ...
Top_N

======================================== 31194 of 56640 ====================================================
Name 		: Agua de Colonia  : Roble y Agave
Brand 		: Sanborns
Gender 		: genderM
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Agua de Colonia  : Roble y Agave is a men's fragrance by Sanborns ...
Top_Notes 	: ['oak', 'agave']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: []

======================================== 31195 of 56640 ====================================================
Name 		: Tycoon
Brand 		: Oriflame
Gender 		: genderM
Average_Rating 	: 3
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Tycoon is a men's fragrance by Oriflame ...
Top_Notes 	: ['marine notes', 'bitter orange', 'lemon', 'sage']
Middle_Notes 	: ['orange blossom', 'cedarwood']
Base_Notes 	: ['leather', 'vetiver', 'black amber']
Reviews 	: []

======================================== 31196 of 56640 ====================================

======================================== 31216 of 56640 ====================================================
Name 		: Tintura Spiritosa
Brand 		: Peccato Originale
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 1
Positive 	: 1
Neutral 	: 0
Negative 	: 0
Describe 	: Tintura Spiritosa is a shared scent launched in 2014 by Peccato Originale ...
Top_Notes 	: ['blackcurrant', 'nutmeg', 'hedione', 'damaskrrose', 'mat?', 'labdanum']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: [['Darvant', 31216, 'Tintura Spiritosa', 'Peccato Originale', 'Positive']]

======================================== 31217 of 56640 ====================================================
Name 		: Midnight Flower
Brand 		: Neil Morris Fragrances
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 3
Positive 	: 2
Neutral 	: 1
Negative 	: 0
Describe 	: Midnight Flower is a shared scent launched in 2007 by Neil Morris Fragrances ...
Top_Notes 	: ['sweet orange', 'aldehydes']
Middle_Notes 	: ['melati blossom', 'ga

======================================== 31235 of 56640 ====================================================
********************error******************

======================================== 31236 of 56640 ====================================================
Name 		: The Wendol
Brand 		: Blackbird
Gender 		: genderS
Average_Rating 	: 3
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: The Wendol is a shared scent launched in 2012 by Blackbird ...
Top_Notes 	: ['vetiver', 'rain', 'dirt', 'moss']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: []

======================================== 31237 of 56640 ====================================================
Name 		: Raw Spirit - Desert Blush
Brand 		: Nomad Two Worlds Foundation
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Part proceeds from sales of Desert Blush will be used to support sustainable Australian indigenous a ...
Top_Notes 	: ['osmanthus', 

...

======================================== 32951 of 56640 ====================================================
Name 		: Far Away Gold
Brand 		: Avon
Gender 		: genderF
Average_Rating 	: 5
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Far Away Gold is a women's perfume launched in 2014 by Avon ...
Top_Notes 	: ['ylang ylang', 'indian jasmine', 'madagascan vanilla']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: []

======================================== 32952 of 56640 ====================================================
Name 		: He Left His Cologne In My Bedroom
Brand 		: Tauerville
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 2
Positive 	: 1
Neutral 	: 1
Negative 	: 0
Describe 	: He Left His Cologne In My Bedroom is a shared scent launched in 2017 by Tauerville ...
Top_Notes 	: ['fresh notes', 'grass', 'green notes', 'bergamot  citrus', 'rosemary', 'musk']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: [['speedracer', 32952, 'He Left His Cologne In My

======================================== 32969 of 56640 ====================================================
Name 		: Kinexium EDT
Brand 		: King of Shaves
Gender 		: genderM
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: An EDT to complement the King of Shaves shaving range. It's bright and fresh, and because it's packa ...
Top_Notes 	: ['bergamot', 'grapefruit', 'lime', 'cypress', 'lavender', 'coriander seed']
Middle_Notes 	: ['oceanic elements', 'tomato leaves', 'plum', 'cassis']
Base_Notes 	: ['precious woods', 'amber', 'moss', 'musk', 'frankincense.']
Reviews 	: []

======================================== 32970 of 56640 ====================================================
Name 		: Tobacco Royal
Brand 		: Precious Liquid
Gender 		: genderS
Average_Rating 	: 0
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: The company says: Tobacco royal starts with a smooth combination of warm spices. The intensity of th ...
Top_N

======================================== 32986 of 56640 ====================================================
Name 		: Bamboo & Green Tea
Brand 		: Amberfig
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Bamboo & Green Tea is a shared scent launched in 2015 by Amberfig ...
Top_Notes 	: ['citrus', 'yuzu', 'grapefruit']
Middle_Notes 	: ['bamboo', 'green tea', 'lemon balm', 'neroli']
Base_Notes 	: ['atlas cedarwood', 'virginia cedarwood', 'patchouli', 'vetiver']
Reviews 	: []

======================================== 32987 of 56640 ====================================================
Name 		: Flora Psychedelica
Brand 		: 4160 Tuesdays
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 1
Positive 	: 1
Neutral 	: 0
Negative 	: 0
Describe 	: The intoxicating aroma of an imaginary flower, set deep in the sun dappled woods on a late spring da ...
Top_Notes 	: ['lily', 'absinthe', 'rose', 'lemon petitgrain', 'jasmine tea', 'rose'

======================================== 33003 of 56640 ====================================================
Name 		: Shuhoor
Brand 		: Louis Cardin
Gender 		: genderF
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Shuhoor is a women's perfume by Louis Cardin ...
Top_Notes 	: ['rosewood', 'spices', 'white tobacco', 'black musk', 'amber']
Middle_Notes 	: []
Base_Notes 	: []
Reviews 	: []

======================================== 33004 of 56640 ====================================================
Name 		: Amber Oud
Brand 		: Floris
Gender 		: genderS
Average_Rating 	: 4
Review_Count 	: 0
Positive 	: 0
Neutral 	: 0
Negative 	: 0
Describe 	: Amber Oud is a shared scent launched in 2014 by Floris ...
Top_Notes 	: ['bergamot', 'frankincense']
Middle_Notes 	: ['oud', 'patchouli', 'rose']
Base_Notes 	: ['amber', 'labdanum', 'oud', 'myrrh', 'vanilla']
Reviews 	: []

======================================== 33005 of 56640 ==============================

KeyboardInterrupt: 